In [1]:
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, ICRS
from astropy.wcs import WCS
from astropy.time import Time
from astropy import units as u
import numpy as np


astr = {
    'axes': np.array([1, 2]), 
    'cd': np.array([[1., 0.], [0., 1.]]), 
    'cdelt': np.array([0.05595291, 0.05595291]), 
    'coord_sys': 'C', 
    'crpix': np.array([1025., 1025.]), 
    'crval': np.array([358.81906128, -26.78497314]), 
    'ctype': ['RA---SIN', 'DEC--SIN'], 
    'dateobs': '2014-07-06T21:09:27.00', 
    'equinox': 2000.0, 
    'known': 1, 
    'latpole': 0.0, 
    'longpole': 180.0, 
    'mjdobs': 56844.88156250352, 
    'naxis': np.array([2048., 2048.]), 
    'projection': 'SIN', 
    'pv1': np.array([  0.,   0.,  90., 180.,   0.]), 
    'pv2': np.array([-3.19716404e-03, -5.42468588e-05]), 
    'radecsys': 'FK5', 
    'reverse': 0, 
    'x0y0': np.array([0., 0.])
}

ra =  np.array([5.49316406, 5.58105469, 5.66894531, 5.75683594, 5.84472656,   5.93261719, 354.99023437, 355.078125, 355.16601562, 355.25390625])
dec = np.array([-36.98188562, -36.98188562, -36.98188562, -36.98188562, -36.98188562, -36.98188562, -37.02859593, -37.02859593, -37.02859593, -37.02859593])

In [2]:
ra

array([  5.49316406,   5.58105469,   5.66894531,   5.75683594,
         5.84472656,   5.93261719, 354.99023437, 355.078125  ,
       355.16601562, 355.25390625])

In [3]:
dec

array([-36.98188562, -36.98188562, -36.98188562, -36.98188562,
       -36.98188562, -36.98188562, -37.02859593, -37.02859593,
       -37.02859593, -37.02859593])

In [4]:
wcs_astr = WCS(naxis  = 2)
wcs_astr.wcs.cdelt = astr['cdelt']
wcs_astr.wcs.ctype = astr['ctype']
wcs_astr.wcs.crpix = astr['crpix']
wcs_astr.wcs.crval = astr['crval']
# You can try and make the WCS and astr be the exact same, but it doesn't seem to make a difference

In [5]:
# Now use world_to_pixel function for WCS objects
x, y = wcs_astr.world_to_pixel(SkyCoord(ra = ra*u.deg, dec = dec*u.deg, frame = 'fk5'))

In [6]:
print(x)
print(y)

[1119.06913292 1120.31530981 1121.56125993 1122.80698062 1124.05246866
 1125.29772141  969.41098983  970.66227641  971.91368835  973.165223  ]
[840.22143338 840.15528355 840.08827236 840.02039994 839.95166646
 839.88207208 841.07564664 841.1129505  841.14938932 841.18496304]


In [7]:
radecs = wcs_astr.pixel_to_world(x, y)
back_ra = radecs.ra.deg
back_dec = radecs.dec.deg

In [8]:
print(ra)
print(back_ra)
print(ra - back_ra)

[  5.49316406   5.58105469   5.66894531   5.75683594   5.84472656
   5.93261719 354.99023437 355.078125   355.16601562 355.25390625]
[  5.49316202   5.58105266   5.66894328   5.75683391   5.84472453
   5.93261517 354.990232   355.07812263 355.16601325 355.25390389]
[2.03526540e-06 2.03297512e-06 2.03069507e-06 2.02842520e-06
 2.02616551e-06 2.02391598e-06 2.37374587e-06 2.37027814e-06
 2.36681984e-06 2.36337092e-06]


In [9]:
print(dec)
print(back_dec)
print(dec - back_dec)

[-36.98188562 -36.98188562 -36.98188562 -36.98188562 -36.98188562
 -36.98188562 -37.02859593 -37.02859593 -37.02859593 -37.02859593]
[-36.98188761 -36.9818876  -36.98188759 -36.98188758 -36.98188757
 -36.98188756 -37.02859893 -37.02859892 -37.02859891 -37.02859891]
[1.98701154e-06 1.97819745e-06 1.96937868e-06 1.96055532e-06
 1.95172733e-06 1.94289479e-06 3.00083724e-06 2.99272519e-06
 2.98460610e-06 2.97648005e-06]


In [10]:
# Let's grab the x and y from IDL
# which was generated by calling apply_astrometry, obs, ra_arr=ra, dec_arr=dec, x_arr=x, y_arr=y,/ad2xy as per healpix_cnv_generate.pro
# Do note this seems to include refraction correction, which I don't think was intended, but likely happened due to a double negative
# However ignoring refraction does not seem to solve the issue at hand, which is the weird off by one error
x_from_idl = np.array([1118.9751548024879, 1120.2205657542779, 1121.4657446623798, 1122.7106885968249, 1123.9553946282024, 1125.1998598276675, 969.36876533986447, 970.61993991521285, 971.87123450196327, 973.12264615552158])
y_from_idl = np.array([840.27188551097856, 840.20574785136114, 840.13874896655682, 840.07088901401949, 840.00216815322847, 839.93258654569411, 841.12601326820197, 841.16330985118441, 841.19974156417231, 841.23530832169058])
idl_radecs = wcs_astr.pixel_to_world(x_from_idl, y_from_idl)
idl_back_ra = idl_radecs.ra.deg
idl_back_dec = idl_radecs.dec.deg

In [11]:
print(ra)
print(idl_back_ra)
print(ra - idl_back_ra)
print(back_ra - idl_back_ra)

[  5.49316406   5.58105469   5.66894531   5.75683594   5.84472656
   5.93261719 354.99023437 355.078125   355.16601562 355.25390625]
[  5.48630729   5.57414017   5.66197265   5.74980473   5.83763642
   5.9254677  354.98741423 355.0752938  355.16317298 355.25105178]
[0.00685677 0.00691452 0.00697266 0.00703121 0.00709014 0.00714949
 0.00282014 0.0028312  0.00284264 0.00285447]
[0.00685473 0.00691248 0.00697063 0.00702918 0.00708812 0.00714747
 0.00281777 0.00282883 0.00284027 0.00285211]


In [12]:
print(dec)
print(idl_back_dec)
print(dec - idl_back_dec)
print(back_dec - idl_back_dec)

[-36.98188562 -36.98188562 -36.98188562 -36.98188562 -36.98188562
 -36.98188562 -37.02859593 -37.02859593 -37.02859593 -37.02859593]
[-36.9793058  -36.97931129 -36.97931685 -36.97932248 -36.97932818
 -36.97933396 -37.02566451 -37.02566632 -37.02566811 -37.0256699 ]
[-0.00257982 -0.00257433 -0.00256877 -0.00256314 -0.00255744 -0.00255166
 -0.00293142 -0.00292961 -0.00292782 -0.00292603]
[-0.0025818  -0.00257631 -0.00257074 -0.0025651  -0.00255939 -0.0025536
 -0.00293442 -0.00293261 -0.00293081 -0.00292901]


In [13]:
# IDL Code, commented out, but here for reference

# ra = double([5.49316406,   5.58105469,   5.66894531,   5.75683594,5.84472656,   5.93261719, 354.99023437, 355.078125  ,355.16601562, 355.25390625])
# dec = double([-36.98188562, -36.98188562, -36.98188562, -36.98188562,-36.98188562, -36.98188562, -37.02859593, -37.02859593,-37.02859593, -37.02859593])
# astr = {
#     "astr",
#     "NAXIS": [2048.0000, 2048.0000],
#     "CD": [1.0000000, 0.0000000, 0.0000000, 1.0000000],
#     "CDELT": [0.055952907, 0.055952907],
#     "CRPIX": [1025.0000, 1025.0000],
#     "CRVAL": [358.81906127929688, -26.784973144531250],
#     "CTYPE": [ "RA---SIN", "DEC--SIN"],
#     "LONGPOLE": 180.00000000000000,
#     "LATPOLE": 0.0000000000000000,
#     "PV2": [-0.0031971640419214964, -5.4246858780970797e-05],
#     "PV1": [0.0000000000000000, 0.0000000000000000, 90.000000000000000, 180.00000000000000, 0.0000000000000000],
#     "AXES": [1, 2],
#     "REVERSE": 0,
#     "COORD_SYS": "C",
#     "PROJECTION": "SIN",
#     "KNOWN": [1],
#     "RADECSYS": "FK5",
#     "EQUINOX": 2000.0000000000000,
#     "DATEOBS": "2014-07-06T21:09:27.00",
#     "MJDOBS": 56844.881562503520,
#     "X0Y0": [0.0000000000000000, 0.0000000000000000]
# }

# By default this INCLUDES refraction, and is how it's used inside healpix_cnv_generate, ideally we should want to ignore refraction
# given we're not doing any optical work here right? (however it's worth noting the difference is not due to refraction being applied)
# IDL> apply_astrometry, obs, ra_arr=pix_ra, dec_arr=pix_dec, x_arr=xv_hpx, y_arr=yv_hpx, /ad2xy
# IDL> x
#    1118.9751548024879       1120.2205657542779       1121.4657446623798       1122.7106885968249       1123.9553946282024       1125.1998598276675
#    969.36876533986447       970.61993991521285       971.87123450196327       973.12264615552158
# IDL> y
#    840.27188551097856       840.20574785136114       840.13874896655682       840.07088901401949       840.00216815322847       839.93258654569411
#    841.12601326820197       841.16330985118441       841.19974156417231       841.23530832169058
# Going back to ra/dec
# IDL> apply_astrometry, obs, ra_arr=back_ra, dec_arr=back_dec, x_arr=x, y_arr=y,/xy2ad
# IDL> back_ra
#        5.4931643977177487       5.5810550271547754       5.6689456565928822       5.7568362860319553       5.8447269154719379       5.9326175449130005
#        354.99023418319234       355.07812481260612       355.16601544201944       355.25390607143220
# IDL> back_dec
#       -36.981884449910950      -36.981884449720653      -36.981884449527975      -36.981884449332682      -36.981884449134853      -36.981884448934544
#       -37.028595424864314      -37.028595424970653      -37.028595425074649      -37.028595425176043
# IDL> ra - back_ra
#   -3.3521774867040222e-07  -3.3965477541642031e-07  -3.4409288218739675e-07  -3.4853195529649383e-07  -3.5297193790029269e-07  -3.5741300052904990e-07
#    1.9180765775672626e-07   1.8739387996902224e-07   1.8298055692866910e-07   1.7856780232250458e-07
# IDL> dec - back_dec
#    4.4722540337716055e-07   4.4703510582166928e-07   4.4684242794801321e-07   4.4664713527708955e-07   4.4644930596859922e-07   4.4624899686596109e-07
#    4.5416118865659882e-07   4.5426752848243268e-07   4.5437152351723853e-07   4.5447291796563150e-07

# So here it is including ignoring refraction (it still doesn't solve the main problem though, but some values are closer as to be expected)

# IDL> apply_astrometry, obs, ra_arr=ra, dec_arr=dec, x_arr=x, y_arr=y,/ad2xy, /ignore_refraction
# IDL> x
#        1119.0016442291212       1120.2474022390218       1121.4929281541154       1122.7382190435615       1123.9832719770679       1125.2280840249075
#        969.35362310327173       970.60514602254648       971.85678897195919       973.10854900627294
# IDL> y
#        840.22033014879560       840.15417303532956       840.08715445229643       840.01927455739349       839.95053351035244       839.88093147292375
#        841.07472441355776       841.11203211984991       841.14847470224345       841.18405207498540
# IDL> apply_astrometry, obs, ra_arr=back_ra, dec_arr=back_dec, x_arr=x, y_arr=y,/xy2ad,/ignore_refraction
# IDL> back_ra
#        5.4931643975646800       5.5810550269996693       5.6689456564355707       5.7568362858725113       5.8447269153103454       5.9326175447493092
#        354.99023418327857       355.07812481269031       355.16601544210158       355.25390607151240
# IDL> back_dec
#       -36.981884449706115      -36.981884449515888      -36.981884449323061      -36.981884449127762      -36.981884448929854      -36.981884448729531
#       -37.028595424659464      -37.028595424765896      -37.028595424869856      -37.028595424971343
# IDL> ra - back_ra  
#   -3.3506468000155110e-07  -3.3949966926627440e-07  -3.4393557069023473e-07  -3.4837251128294611e-07  -3.5281034538314771e-07  -3.5724930924629916e-07
#    1.9172142629031441e-07   1.8730969486568938e-07   1.8289841818841523e-07   1.7848759625849198e-07
# IDL> dec - back_dec
#    4.4702056811729562e-07   4.4683034161607793e-07   4.4663751452844735e-07   4.4644221475209633e-07   4.4624430728390507e-07   4.4604398397041223e-07
#    4.5395633918587919e-07   4.5406277138226869e-07   4.5416673088993775e-07   4.5426821770888637e-07
